Big shoutout to: https://www.kaggle.com/marissafernandes
I am currently a beginner and his kernel helped me figure out valuable information about this dataset.

However, my kernel differ from his kernel by the usage of 'H'.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../input/asteroid-dataset/dataset.csv', index_col = 'pdes')

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
cols_with_missing_data = [col for col in df.columns
                         if df[col].isnull().any()]
missing_data_count = df[cols_with_missing_data].isnull().sum()
print(missing_data_count)

In [ ]:
df_input = df.dropna(axis=0,how='any',subset=['pha']).copy()

In [ ]:
def print_cols_with_missing_data():
    cols_with_missing_data = [col for col in df_input.columns
                             if df_input[col].isnull().any()]
    missing_data_count = df_input[cols_with_missing_data].isnull().sum()
    print(missing_data_count)

In [ ]:
df_input[df_input['neo'].isnull()]

You can look up these asteroid on: https://ssd.jpl.nasa.gov/sbdb.cgi
Based on their moid and orbital parameters, it clear these are not near-earth objects.

In [ ]:
print_cols_with_missing_data()

df_input['neo'] = df_input['neo'].fillna('N')
df_input = df_input.dropna(axis=0, subset=['sigma_ad'])
df_input = df_input.dropna(axis=0, subset=['ma'])

There are 1->4 problematic indexes, as they do not carry much statistic signicficant, its best to remove them than try to imputer their values.

There statistic significant is determined subjectively as their's moid is all higher than 0.05 au meaning: pha = N

In [ ]:
print_cols_with_missing_data()

The columns with missing values are:
- name, prefix: there are identification, their values already stored in pdes => can be removed
- H: absolute magnitude play a role in determining pha 
- diameter, albedo and diameter_sigma: correlated with H, however, as more of 80% of data is missing, it best to remove these columns

In [ ]:
df[(df['pha']=='Y') & (df['H'].isnull())].info()

This shown that all data with 'pha' = 'Y' have a known 'H'
-> removing valua without 'H' is possible as this does not affect our interested feature.

In [ ]:
df_input = df_input.dropna(axis=0, subset=['H'])

In [ ]:
print_cols_with_missing_data()

In [ ]:
drop_cols = ['id', 'spkid', 'name', 'full_name', 'prefix', 'equinox', 
             'diameter', 'albedo', 'diameter_sigma', 'class', 'orbit_id']
df_input = df_input.drop(drop_cols, axis=1)

In [ ]:
df_input['neo'] = df_input['neo'].replace(('Y','N'), (1,0))
df_input['pha'] = df_input['pha'].replace(('Y','N'), (1,0))

In [ ]:
df_input.info()

We arrived at a full numerical dataframe with no missing values

In [ ]:
df_output = df_input['pha'].copy()
df_input = df_input.drop(['pha'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(df_input, df_output, 
                                                      train_size = 0.8, test_size = 0.2) 

In [ ]:
X_train.head()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler

# Scaler
scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_train = pd.DataFrame(scaled_X_train, columns = X_train.columns, index = X_train.index)
scaled_X_valid = scaler.transform(X_valid)
scaled_X_valid = pd.DataFrame(scaled_X_valid, columns = X_valid.columns, index = X_valid.index)

In [ ]:
scaled_X_train.head()

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE()
X_train_res, y_train_res = sm.fit_resample(scaled_X_train, y_train.ravel()) 

In [ ]:
print('Number of PHA = Y after resampling {}'.format(sum(y_train_res == 1)))
print('Number of PHA = N after resampling {}'.format(sum(y_train_res == 0)))

In [ ]:
X_train_res.head()

In [ ]:
from sklearn import metrics
def metricCalculation(y_test, pred):
    
    precision_metric = metrics.precision_score(y_test, pred, average = "macro")
    recall_metric = metrics.recall_score(y_test, pred, average = "macro")
    accuracy_metric = metrics.accuracy_score(y_test, pred)
    f1_metric = metrics.f1_score(y_test, pred, average = "macro")
    print('Precision metric:',round(precision_metric, 8))
    print('Recall Metric:',round(recall_metric, 8))
    print('Accuracy Metric:',round(accuracy_metric, 8))
    print('F1 score:',round(f1_metric, 8))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train_res, y_train_res)
pred = model.predict(scaled_X_valid)
metricCalculation(y_valid, pred)

In [ ]:
metrics.confusion_matrix(y_valid, pred)

In [ ]:
feature_imp = pd.DataFrame(model.feature_importances_,
                           index=X_train_res.columns,
                           columns = ['Importance']).sort_values(by='Importance', ascending=False)
print(feature_imp)

The result indicate that moid_ld and H are the most importance features - this is expected according to multiple online sources.
I suspect diameter also play a big role (as it is directly correlate to distance and H).

However, given that more than 80% of diameter data is missing, it best to delete the columns entirely.

Finetune comments:
- Deleting columns such as neo, sigma of orbital parameters is possible -> achiving a more accurate model (removing data noise)
- it is possible to calculate some missing value of H, diameter and albedo (when 1 unknown and 2 known)using: http://www.physics.sfasu.edu/astro/asteroids/sizemagnitude.html . However, the amound of data retrieved using this method is minimal. Also, the value is also slightly different from the dataset, for example, pdes == 1, Ceres, whose diameter calculated using given online resource is 954, whereas the dataset gave us 939.

I will not finetune these feature, as they serve little purpose for learning, feel free to finetune these features yourself.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier

models = [LogisticRegression(max_iter = 5000), LinearSVC(), DecisionTreeClassifier(),
          RandomForestClassifier(), KNeighborsClassifier()]

- LogisticRegression, Linear SVC, and KNeighborsClassifier achieve terrible F1 score.
- RandomForest() have a long running time
- DecisionTree have good result - and minimal running time compared to RandomForest

Feel free to add models as you like.

In [ ]:
for model in models:
    model.fit(X_train_res, y_train_res)
    pred = model.predict(scaled_X_valid)
    metricCalculation(y_valid, pred)